In [1]:
import copy
import time
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings('ignore')

from Function import Window_LogReg,Window_pca,Window_rf,Window_KNN,Model_return

In [2]:
folderpath = 'C:/Users/deku2/PycharmProjects/AppliedProject/Processing/Processed_data/'
sentiment = pd.read_csv(folderpath + 'Processed_bitcoin_2017-12-01_2021-07-01_withlags_minmax.csv')
sentiment = sentiment.rename(columns={"created_at": "Date"})
sentiment = sentiment.set_index('Date')
price = pd.read_csv(folderpath + 'Processed_BTC-USD_withlags_3%.csv')
price = price.set_index('Date')
data = pd.concat([sentiment,price], axis=1)

In [3]:
X_train = data.drop(columns=['State','State_tmr'])[0:1096]
Y_train = data['State_tmr'][0:1096]
Return_train = X_train['Return']
X_test = data.drop(columns=['State','State_tmr'])
Y_test = data['State_tmr']
seed = 123
np.random.seed(seed)
performance_measures = ['precision', 'recall', 'f1', 'roc_auc', 'accuracy']

In [4]:
y_test = [1]*len(Return_train[912:])
baseline = Model_return(Return_train[912:], y_test)
baseline[-1]

3.1737516304187077

In [9]:
Return_test = X_test['Return']
y_test = [1]*len(Return_test[1096:])
baseline = Model_return(Return_test[1096:], y_test)
baseline[-1]

1.2082329975122945

In [8]:
# Set performance to Performance(Y_train[1096:], y_pred[1096:])
# X_train[912:]

In [9]:
#State tmr moving window better
columns = ['parameter', 'windows'] + performance_measures + ['model_return']
results = []
model_returns = []
parameter = ['l2']
windows = [180,365,730]
for para in parameter:
    for window in windows:
        print('parameter =',para, '\twindow = ', window)
        start = time.time()
        result, model_return = Window_LogReg(X_train, Y_train, Return_train, window, 1, 0, para)
        end = time.time()
        results.append([para,window] + result + [model_return[-1]])
        model_returns.append(model_return)
        print('time taken = ', np.round((end - start)/60, 1), 'min', result)
logreg_under = pd.DataFrame(results,columns=columns)
logreg_under.to_csv(f'standard/logreg_under.csv')
logreg_under_returns = pd.DataFrame(model_returns)
logreg_under_returns.to_csv(f'standard/logreg_under_returns.csv')

parameter = l2 	window =  180
time taken =  0.3 min [0.2247191011235955, 0.7142857142857143, 0.3418803418803419, 0.635989010989011, 0.5815217391304348]
parameter = l2 	window =  365
time taken =  0.2 min [0.10714285714285714, 0.10714285714285714, 0.10714285714285714, 0.4734432234432235, 0.7282608695652174]
parameter = l2 	window =  730
time taken =  0.1 min [0.15384615384615385, 0.07142857142857142, 0.0975609756097561, 0.5004578754578755, 0.7989130434782609]


In [10]:
columns = ['parameter', 'windows'] + performance_measures + ['model_return']
results = []
model_returns = []
parameter = ['l2']
windows = [180,365,730]
for para in parameter:
    for window in windows:
        print('parameter =',para, '\twindow = ', window)
        start = time.time()
        result, model_return = Window_LogReg(X_train, Y_train, Return_train, window, 1, 1, para)
        end = time.time()
        results.append([para,window] + result + [model_return[-1]])
        model_returns.append(model_return)
        print('time taken = ', np.round((end - start)/60, 1), 'min', result)
logreg_over = pd.DataFrame(results,columns=columns)
logreg_over.to_csv(f'standard/logreg_over.csv')
logreg_over_returns = pd.DataFrame(model_returns)
logreg_over_returns.to_csv(f'standard/logreg_over_returns.csv')

parameter = l2 	window =  180
time taken =  0.3 min [0.26153846153846155, 0.6071428571428571, 0.3655913978494623, 0.6497252747252747, 0.6793478260869565]
parameter = l2 	window =  365
time taken =  0.3 min [0.1951219512195122, 0.2857142857142857, 0.2318840579710145, 0.5370879120879121, 0.7119565217391305]
parameter = l2 	window =  730
time taken =  0.2 min [0.17525773195876287, 0.6071428571428571, 0.272, 0.5471611721611722, 0.5054347826086957]


In [11]:
columns = ['parameter', 'PCA', 'windows'] + performance_measures + ['model_return']
results = []
model_returns = []
parameter = ['l2']
pca_para = [2, 5, 10, 20]
windows = [180,365,730]
for para in parameter:
    for pca in pca_para:
        for window in windows:
            print('parameter = ',para, '\tPCA = ',pca, '\twindow = ', window)
            start = time.time()
            result, model_return = Window_pca(X_train, Y_train, Return_train, window, 1, 0, pca, para)
            end = time.time()
            results.append([para,pca,window] + result + [model_return[-1]])
            model_returns.append(model_return)
            print('time taken = ', np.round((end - start)/60, 1), 'min', result)
pca_under = pd.DataFrame(results,columns=columns)
pca_under.to_csv(f'standard/pca_under.csv')
pca_under_returns = pd.DataFrame(model_returns)
pca_under_returns.to_csv(f'standard/pca_under_returns.csv')

parameter =  l2 	PCA =  2 	window =  180
time taken =  0.3 min [0.22988505747126436, 0.7142857142857143, 0.3478260869565217, 0.6423992673992674, 0.592391304347826]
parameter =  l2 	PCA =  2 	window =  365
time taken =  0.2 min [0.15789473684210525, 0.21428571428571427, 0.18181818181818182, 0.5045787545787546, 0.7065217391304348]
parameter =  l2 	PCA =  2 	window =  730
time taken =  0.1 min [0.17647058823529413, 0.10714285714285714, 0.13333333333333333, 0.5086996336996337, 0.7880434782608695]
parameter =  l2 	PCA =  5 	window =  180
time taken =  0.3 min [0.21739130434782608, 0.7142857142857143, 0.3333333333333333, 0.6263736263736264, 0.5652173913043478]
parameter =  l2 	PCA =  5 	window =  365
time taken =  0.2 min [0.10714285714285714, 0.10714285714285714, 0.10714285714285714, 0.4734432234432235, 0.7282608695652174]
parameter =  l2 	PCA =  5 	window =  730
time taken =  0.1 min [0.2, 0.07142857142857142, 0.10526315789473682, 0.51007326007326, 0.8152173913043478]
parameter =  l2 	PCA 

In [12]:
columns = ['parameter', 'PCA', 'windows'] + performance_measures + ['model_return']
results = []
model_returns = []
parameter = ['l2']
pca_para = [2, 5, 10, 20, 30]
windows = [180,365,730]
for para in parameter:
    for pca in pca_para:
        for window in windows:
            print('parameter = ',para, '\tPCA = ',pca, '\twindow = ', window)
            start = time.time()
            result, model_return = Window_pca(X_train, Y_train, Return_train, window, 1, 1, pca, para)
            end = time.time()
            results.append([para,pca,window] + result + [model_return[-1]])
            model_returns.append(model_return)
            print('time taken = ', np.round((end - start)/60, 1), 'min', result)
pca_over = pd.DataFrame(results,columns=columns)
pca_over.to_csv(f'standard/pca_over.csv')
pca_over_returns = pd.DataFrame(model_returns)
pca_over_returns.to_csv(f'standard/pca_over_returns.csv')

parameter =  l2 	PCA =  2 	window =  180
time taken =  0.3 min [0.2261904761904762, 0.6785714285714286, 0.3392857142857143, 0.6309523809523809, 0.5978260869565217]
parameter =  l2 	PCA =  2 	window =  365
time taken =  0.3 min [0.17105263157894737, 0.4642857142857143, 0.25, 0.5302197802197803, 0.5760869565217391]
parameter =  l2 	PCA =  2 	window =  730
time taken =  0.2 min [0.17, 0.6071428571428571, 0.265625, 0.5375457875457875, 0.4891304347826087]
parameter =  l2 	PCA =  5 	window =  180
time taken =  0.3 min [0.25316455696202533, 0.7142857142857143, 0.3738317757009346, 0.668040293040293, 0.6358695652173914]
parameter =  l2 	PCA =  5 	window =  365
time taken =  0.3 min [0.2127659574468085, 0.35714285714285715, 0.26666666666666666, 0.559981684981685, 0.7010869565217391]
parameter =  l2 	PCA =  5 	window =  730
time taken =  0.2 min [0.17699115044247787, 0.7142857142857143, 0.28368794326241137, 0.5590659340659341, 0.45108695652173914]
parameter =  l2 	PCA =  10 	window =  180
time ta

In [14]:
columns = ['n_neighbors', 'windows'] + performance_measures + ['model_return']
results = []
model_returns = []
n_neighbors = [5, 10, 15, 20, 25]
windows = [180,365,730]
for n in n_neighbors:
    for window in windows:
        print('n_neighbors = ',n,'window = ', window)
        start = time.time()
        result, model_return = Window_KNN(X_train, Y_train, Return_train, window, 1, 0, n)
        end = time.time()
        results.append([n,window] + result + [model_return[-1]])
        model_returns.append(model_return)
        print('time taken = ', np.round((end - start)/60, 1), 'min', result)
KNN_under = pd.DataFrame(results,columns=columns)
KNN_under.to_csv(f'standard/KNN_under.csv')
KNN_under_returns = pd.DataFrame(model_returns)
KNN_under_returns.to_csv(f'standard/KNN_under_returns.csv')

n_neighbors =  5 window =  180
time taken =  0.3 min [0.18867924528301888, 0.35714285714285715, 0.2469135802469136, 0.5407509157509158, 0.6684782608695652]
n_neighbors =  5 window =  365
time taken =  0.3 min [0.20930232558139536, 0.32142857142857145, 0.25352112676056343, 0.5517399267399267, 0.7119565217391305]
n_neighbors =  5 window =  730
time taken =  0.1 min [0.21428571428571427, 0.32142857142857145, 0.2571428571428572, 0.5549450549450549, 0.717391304347826]
n_neighbors =  10 window =  180
time taken =  0.3 min [0.2, 0.25, 0.22222222222222224, 0.5352564102564102, 0.7336956521739131]
n_neighbors =  10 window =  365
time taken =  0.3 min [0.14814814814814814, 0.14285714285714285, 0.14545454545454545, 0.49771062271062266, 0.7445652173913043]
n_neighbors =  10 window =  730
time taken =  0.1 min [0.21212121212121213, 0.25, 0.22950819672131145, 0.5416666666666667, 0.7445652173913043]
n_neighbors =  15 window =  180
time taken =  0.3 min [0.28205128205128205, 0.39285714285714285, 0.3283

In [15]:
columns = ['n_neighbors', 'windows'] + performance_measures + ['model_return']
results = []
model_returns = []
n_neighbors = [10, 20, 50, 100, 200]
windows = [180,365,730]
for n in n_neighbors:
    for window in windows:
        print('n_neighbors = ',n,'window = ', window)
        start = time.time()
        result, model_return = Window_KNN(X_train, Y_train, Return_train, window, 1, 1, n)
        end = time.time()
        results.append([n,window] + result + [model_return[-1]])
        model_returns.append(model_return)
        print('time taken = ', np.round((end - start)/60, 1), 'min', result)
KNN_over = pd.DataFrame(results,columns=columns)
KNN_over.to_csv(f'standard/KNN_over.csv')
KNN_over_returns = pd.DataFrame(model_returns)
KNN_over_returns.to_csv(f'standard/KNN_over_returns.csv')

n_neighbors =  10 window =  180
time taken =  0.3 min [0.16842105263157894, 0.5714285714285714, 0.2601626016260163, 0.5325091575091575, 0.5054347826086957]
n_neighbors =  10 window =  365
time taken =  0.3 min [0.14583333333333334, 0.5, 0.22580645161290322, 0.4871794871794871, 0.4782608695652174]
n_neighbors =  10 window =  730
time taken =  0.1 min [0.1559633027522936, 0.6071428571428571, 0.24817518248175183, 0.5086996336996337, 0.44021739130434784]
n_neighbors =  20 window =  180
time taken =  0.3 min [0.19008264462809918, 0.8214285714285714, 0.3087248322147651, 0.5966117216117216, 0.44021739130434784]
n_neighbors =  20 window =  365
time taken =  0.3 min [0.1388888888888889, 0.5357142857142857, 0.22058823529411764, 0.4697802197802198, 0.42391304347826086]
n_neighbors =  20 window =  730
time taken =  0.1 min [0.17829457364341086, 0.8214285714285714, 0.29299363057324845, 0.5709706959706959, 0.3967391304347826]
n_neighbors =  50 window =  180
time taken =  0.3 min [0.19402985074626866

In [17]:
columns = ['n_estimators', 'max_depth', 'windows'] + performance_measures + ['model_return']
results = []
model_returns = []
n_estimators = [10, 20, 50, 100]
max_depth = [10, 20]
windows = [180,365,730]
for n in n_estimators:
    for m_depth in max_depth:
        for window in windows:
            print('n_estimators = ',n, '\tmax_depth = ',m_depth, '\twindow = ', window)
            start = time.time()
            result, model_return = Window_rf(X_train, Y_train, Return_train, window, 1, 0, n, m_depth)
            end = time.time()
            results.append([n,m_depth,window] + result + [model_return[-1]])
            model_returns.append(model_return)
            print('time taken = ', np.round((end - start)/60, 1), 'min', result)
rf_under = pd.DataFrame(results,columns=columns)
rf_under.to_csv(f'standard/rf_under.csv')
rf_under_returns = pd.DataFrame(model_returns)
rf_under_returns.to_csv(f'standard/rf_under_returns.csv')

n_estimators =  10 	max_depth =  10 	window =  180
time taken =  0.6 min [0.21621621621621623, 0.5714285714285714, 0.3137254901960784, 0.5998168498168498, 0.6195652173913043]
n_estimators =  10 	max_depth =  10 	window =  365
time taken =  0.5 min [0.23376623376623376, 0.6428571428571429, 0.34285714285714286, 0.6323260073260073, 0.625]
n_estimators =  10 	max_depth =  10 	window =  730
time taken =  0.3 min [0.21052631578947367, 0.5714285714285714, 0.3076923076923077, 0.5934065934065934, 0.6086956521739131]
n_estimators =  10 	max_depth =  20 	window =  180
time taken =  0.7 min [0.21621621621621623, 0.5714285714285714, 0.3137254901960784, 0.5998168498168498, 0.6195652173913043]
n_estimators =  10 	max_depth =  20 	window =  365
time taken =  0.6 min [0.23376623376623376, 0.6428571428571429, 0.34285714285714286, 0.6323260073260073, 0.625]
n_estimators =  10 	max_depth =  20 	window =  730
time taken =  0.3 min [0.22058823529411764, 0.5357142857142857, 0.3125, 0.597985347985348, 0.64130

In [18]:
columns = ['n_estimators', 'max_depth', 'windows'] + performance_measures + ['model_return']
results = []
model_returns = []
n_estimators = [10, 20, 50, 100]
max_depth = [10, 20]
windows = [180,365,730]
for n in n_estimators:
    for m_depth in max_depth:
        for window in windows:
            print('n_estimators = ',n, '\tmax_depth = ',m_depth, '\twindow = ', window)
            start = time.time()
            result, model_return = Window_rf(X_train, Y_train, Return_train, window, 1, 1, n, m_depth)
            end = time.time()
            results.append([n,m_depth,window] + result + [model_return[-1]])
            model_returns.append(model_return)
            print('time taken = ', np.round((end - start)/60, 1), 'min', result)
rf_over = pd.DataFrame(results,columns=columns)
rf_over.to_csv(f'standard/rf_over.csv')
rf_over_returns = pd.DataFrame(model_returns)
rf_over_returns.to_csv(f'standard/rf_over_returns.csv')

n_estimators =  10 	max_depth =  10 	window =  180
time taken =  0.7 min [0.0, 0.0, 0.0, 0.46474358974358976, 0.7880434782608695]
n_estimators =  10 	max_depth =  10 	window =  365
time taken =  0.6 min [0.058823529411764705, 0.03571428571428571, 0.044444444444444446, 0.46657509157509164, 0.7663043478260869]
n_estimators =  10 	max_depth =  10 	window =  730
time taken =  0.4 min [0.125, 0.14285714285714285, 0.13333333333333333, 0.48168498168498164, 0.717391304347826]
n_estimators =  10 	max_depth =  20 	window =  180
time taken =  0.8 min [0.18181818181818182, 0.07142857142857142, 0.10256410256410255, 0.5068681318681318, 0.8097826086956522]
n_estimators =  10 	max_depth =  20 	window =  365
time taken =  0.6 min [0.23529411764705882, 0.14285714285714285, 0.17777777777777778, 0.5297619047619047, 0.7989130434782609]
n_estimators =  10 	max_depth =  20 	window =  730
time taken =  0.4 min [0.18181818181818182, 0.07142857142857142, 0.10256410256410255, 0.5068681318681318, 0.80978260869565